# Face search using Amazon Rekognition

***
This notebook provides a walkthrough of [compare faces API]( https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CompareFaces.html) and [face search API](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_SearchFaces.html) in Amazon Rekognition to detect known faces.
***

In [3]:
# Initialize dependencies
import boto3
import botocore
from IPython.display import HTML, display, Image as IImage
import time

# Initialize clients
REGION = boto3.session.Session().region_name or 'us-east-1'
rekognition = boto3.client('rekognition', REGION)
s3 = boto3.client('s3', REGION)

bucket_name = 'test-images-891377001208'

# Detect known faces in image

There are two main ways to detect known faces by using Amazon Rekognition.


1. The first is by using [the compare faces API](https://docs.aws.amazon.com/rekognition/latest/dg/faces-comparefaces.html) to compare two images.
2. The second is by creating a collection of known images using the [index faces API and then using the search faces API](https://docs.aws.amazon.com/rekognition/latest/dg/collections.html) to detect all the faces belonging to a particular collection on an image or video.
***

### 1. Call Rekognition to compare two faces

<https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CompareFaces.html>

In [5]:
# Show images
source = "identity.jpg" # image profile
target = "looking_at_screen.jpg" # screenshot

display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': source})))
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': target})))

In [6]:
compare_faces_response = rekognition.compare_faces(
    SourceImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': source
        }
    },
    TargetImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': target
        }        
    })

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see detected faces, confidence score and additional information.

In [7]:
print("Matched Faces: {}".format(len(compare_faces_response['FaceMatches'])))
display(compare_faces_response)

Matched Faces: 1


{'SourceImageFace': {'BoundingBox': {'Width': 0.4096868336200714,
   'Height': 0.40416184067726135,
   'Left': 0.2767603099346161,
   'Top': 0.1920318752527237},
  'Confidence': 99.99990844726562},
 'FaceMatches': [{'Similarity': 99.9990005493164,
   'Face': {'BoundingBox': {'Width': 0.1543322205543518,
     'Height': 0.38393434882164,
     'Left': 0.44668450951576233,
     'Top': 0.20054830610752106},
    'Confidence': 99.99980926513672,
    'Landmarks': [{'Type': 'eyeLeft',
      'X': 0.4895631968975067,
      'Y': 0.35889360308647156},
     {'Type': 'eyeRight', 'X': 0.5552297830581665, 'Y': 0.3527182638645172},
     {'Type': 'mouthLeft', 'X': 0.49813297390937805, 'Y': 0.48312491178512573},
     {'Type': 'mouthRight', 'X': 0.5526813268661499, 'Y': 0.47797098755836487},
     {'Type': 'nose', 'X': 0.5225225687026978, 'Y': 0.41206860542297363}],
    'Pose': {'Roll': -4.344897270202637,
     'Yaw': -0.6061569452285767,
     'Pitch': 12.328144073486328},
    'Quality': {'Brightness': 65.2

### 2. Use Index Faces and Search Faces

Now let's detect faces using the Search functionality. 

#### Call Rekognition to create a new collection

<https://docs.aws.amazon.com/rekognition/latest/dg/create-collection-procedure.html>

In [8]:
collection_id = bucket_name

# Create collection unless it already exsists
try:
    rekognition.describe_collection(CollectionId=collection_id)
    print("Collection already exists")
except botocore.exceptions.ClientError as e:
    rekognition.create_collection(CollectionId=collection_id)
    print("Collection created")

Collection created


In [10]:
# Show image
image_name = "looking_at_screen.jpg"
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': image_name})))

#### Call Rekognition to search faces in the collection

By making this call, unsurpsingly we'll get no matches the first time, because no image has been indexed yet.

<https://docs.aws.amazon.com/rekognition/latest/dg/search-face-with-image-procedure.html>

In [11]:
search_faces_response = rekognition.search_faces_by_image(
    CollectionId=collection_id,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see detected faces, confidence score and additional information.

In [12]:
print("Matched Faces: {}".format(len(search_faces_response['FaceMatches'])))
display(search_faces_response)

Matched Faces: 0


{'SearchedFaceBoundingBox': {'Width': 0.1543322205543518,
  'Height': 0.38393434882164,
  'Left': 0.44668450951576233,
  'Top': 0.20054830610752106},
 'SearchedFaceConfidence': 99.99980926513672,
 'FaceMatches': [],
 'FaceModelVersion': '7.0',
 'ResponseMetadata': {'RequestId': 'f3a3eb87-9a20-478b-b457-91bbda8911ec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f3a3eb87-9a20-478b-b457-91bbda8911ec',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '220',
   'date': 'Thu, 12 Dec 2024 18:03:44 GMT'},
  'RetryAttempts': 0}}

#### Index Face in the Collection

Now let's call Amazon Rekognition to create a new identity for the given collection.
<https://docs.aws.amazon.com/rekognition/latest/APIReference/API_IndexFaces.html>

In [14]:
index_faces_response = rekognition.index_faces(
    CollectionId=collection_id,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': 'identity.jpg',
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see an ID identifying the newly created identity called FaceId.
You can persist that ID in a Database and associate it with a Name for a future lookup.

In [15]:
identities = {}
identities[index_faces_response['FaceRecords'][0]['Face']['FaceId']] = 'John Doe'
display(index_faces_response)

{'FaceRecords': [{'Face': {'FaceId': '17b14ca4-916f-4d68-b7a5-9de7155f6ebe',
    'BoundingBox': {'Width': 0.4096868336200714,
     'Height': 0.40416184067726135,
     'Left': 0.2767603099346161,
     'Top': 0.1920318752527237},
    'ImageId': 'a112d078-7e86-3fd0-bd81-cbe3c3a825af',
    'Confidence': 99.99990844726562},
   'FaceDetail': {'BoundingBox': {'Width': 0.4096868336200714,
     'Height': 0.40416184067726135,
     'Left': 0.2767603099346161,
     'Top': 0.1920318752527237},
    'Landmarks': [{'Type': 'eyeLeft',
      'X': 0.38909125328063965,
      'Y': 0.35645154118537903},
     {'Type': 'eyeRight', 'X': 0.5686460137367249, 'Y': 0.3490675687789917},
     {'Type': 'mouthLeft', 'X': 0.41466090083122253, 'Y': 0.4943619668483734},
     {'Type': 'mouthRight', 'X': 0.5641433000564575, 'Y': 0.4880884885787964},
     {'Type': 'nose', 'X': 0.4862532615661621, 'Y': 0.41749873757362366}],
    'Pose': {'Roll': -2.4268858432769775,
     'Yaw': 0.20838695764541626,
     'Pitch': 10.606622695

#### Call Rekognition again to search faces in the collection

This time, the search should match the indexed identity.
<https://docs.aws.amazon.com/rekognition/latest/dg/search-face-with-image-procedure.html>

In [16]:
search_faces_response = rekognition.search_faces_by_image(
    CollectionId=collection_id,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see detected faces, confidence score and additional information.

In [17]:
print("Matched Faces: {}".format(len(search_faces_response['FaceMatches'])))
for match in search_faces_response['FaceMatches']:
    print("Detected {} with Confidence: {}".format(identities[match['Face']['FaceId']], match['Face']['Confidence']))

display(search_faces_response)

Matched Faces: 1
Detected John Doe with Confidence: 99.9999008178711


{'SearchedFaceBoundingBox': {'Width': 0.1543322205543518,
  'Height': 0.38393434882164,
  'Left': 0.44668450951576233,
  'Top': 0.20054830610752106},
 'SearchedFaceConfidence': 99.99980926513672,
 'FaceMatches': [{'Similarity': 99.9990005493164,
   'Face': {'FaceId': '17b14ca4-916f-4d68-b7a5-9de7155f6ebe',
    'BoundingBox': {'Width': 0.40968701243400574,
     'Height': 0.4041619896888733,
     'Left': 0.2767600119113922,
     'Top': 0.19203199446201324},
    'ImageId': 'a112d078-7e86-3fd0-bd81-cbe3c3a825af',
    'Confidence': 99.9999008178711,
    'IndexFacesModelVersion': '7.0'}}],
 'FaceModelVersion': '7.0',
 'ResponseMetadata': {'RequestId': '78860856-b7fb-4788-ad5c-7e32dce7cfee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '78860856-b7fb-4788-ad5c-7e32dce7cfee',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '542',
   'date': 'Thu, 12 Dec 2024 18:04:00 GMT'},
  'RetryAttempts': 0}}

# Search faces in video
Search recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/procedure-person-search-videos.html. 

- First we start a search detection job which returns a Job Id.
- We can then call `get_search_detection` to get the job status and after job is complete, we can get face metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

In [18]:
video_name = "leaving.mp4"

#### Show video in the player

In [19]:
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': video_name})

video_tag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3_video_url)
video_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(video_tag)
display(HTML(video_ui))


""


#### Call Rekognition to start a job for face search

In [20]:
start_search_detection = rekognition.start_face_search(
    CollectionId=collection_id,
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': video_name,
        }
    },
)

search_job_id = start_search_detection['JobId']
display("Job Id: {0}".format(search_job_id))

'Job Id: d3c9e426aeedf4281e8afcc34920204466ea145d1b48fc4a8a45f1dd591f10f4'

### Additional (Optional) Request Attributes

ClientRequestToken, FaceMatchThreshold, JobTag, NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceSearch.html


#### Wait for object detection job to complete

In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [21]:
get_search_detection = rekognition.get_face_search(
    JobId=search_job_id
)

while(get_search_detection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    get_search_detection = rekognition.get_face_search(
        JobId=search_job_id)
    
display(get_search_detection['JobStatus'])

...

'SUCCEEDED'

#### Review raw JSON reponse from Rekognition

In the JSON response below, you will see list of detected faces and attributes.
For each detected face, you will see information like Timestamp.

In [22]:
display(get_search_detection)

{'JobStatus': 'SUCCEEDED',
 'VideoMetadata': {'Codec': 'h264',
  'DurationMillis': 8274,
  'Format': 'QuickTime / MOV',
  'FrameRate': 29.970029830932617,
  'FrameHeight': 720,
  'FrameWidth': 1280,
  'ColorRange': 'LIMITED'},
 'Persons': [{'Timestamp': 0,
   'Person': {'Index': 0,
    'Face': {'BoundingBox': {'Width': 0.16818752884864807,
      'Height': 0.4147394895553589,
      'Left': 0.4027940630912781,
      'Top': 0.19215774536132812},
     'Landmarks': [{'Type': 'eyeLeft',
       'X': 0.44816869497299194,
       'Y': 0.3532431721687317},
      {'Type': 'eyeRight', 'X': 0.5202697515487671, 'Y': 0.3480505347251892},
      {'Type': 'mouthLeft', 'X': 0.4570233225822449, 'Y': 0.4924316704273224},
      {'Type': 'mouthRight', 'X': 0.516939103603363, 'Y': 0.4880813956260681},
      {'Type': 'nose', 'X': 0.4849218726158142, 'Y': 0.4130924344062805}],
     'Pose': {'Roll': -3.3523330688476562,
      'Yaw': -1.0463366508483887,
      'Pitch': 11.794028282165527},
     'Quality': {'Bright

#### Display face detected by timestamp and alert when faces are not detected

In [23]:
# Faces detected in each frame
prev_ts = 0
threshold = 1000 # ms
for person in get_search_detection['Persons']:
    ts = person["Timestamp"]
    if ts-prev_ts>threshold:
        print("ALERT - no face matched for {} seconds".format((ts-prev_ts)/1000))
    for match in person["FaceMatches"]:
        confidence = match["Face"]["Confidence"]
        identity = identities[match["Face"]["FaceId"]]
        print("Detected {} on Timestamp: {} with confidence: {}".format(identity, ts, confidence))
    prev_ts = ts

Detected John Doe on Timestamp: 0 with confidence: 99.9999008178711
Detected John Doe on Timestamp: 467 with confidence: 99.9999008178711
Detected John Doe on Timestamp: 967 with confidence: 99.9999008178711
Detected John Doe on Timestamp: 1468 with confidence: 99.9999008178711
Detected John Doe on Timestamp: 1968 with confidence: 99.9999008178711
ALERT - no face matched for 5.506 seconds
Detected John Doe on Timestamp: 7474 with confidence: 99.9999008178711
Detected John Doe on Timestamp: 7974 with confidence: 99.9999008178711


#### Cleanup resources

Let's call Rekognition to delete the Collection.
<https://docs.aws.amazon.com/rekognition/latest/dg/delete-collection-procedure.html>

In [ ]:
rekognition.delete_collection(CollectionId=collection_id)

***
### References
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CreateCollection.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_IndexFaces.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_SearchFaces.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceSearch.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetLabelDetection.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetFaceSearch.html
***

You have successfully used Amazon Rekognition to search for known faces in images and videos.